In [ ]:
from datetime import date

from openfisca_france import init_country
from openfisca_france.model.base import *

# Système socio-fiscal

In [ ]:
TaxBenefitSystem = init_country()
tax_benefit_system = TaxBenefitSystem()

In [ ]:
from openfisca_core import reforms

In [ ]:
ReformeRevenuDeBase = reforms.make_reform(
    key = 'reforme_rdb',
    name = u"Réforme Revenu de base",
    reference = tax_benefit_system,
    )

# Réforme : 1. Revenu de base enfant

In [ ]:
# Suppression des allocations familiales

class af(ReformeRevenuDeBase.Variable):
    reference = Familles.column_by_name['af']

    def function(self, simulation, period):
        period = period.this_month
        af_base = simulation.calculate('af_base', period)
        af_majo = simulation.calculate('af_majo', period)
        af_forf = simulation.calculate('af_forf', period)

        #return period, af_base + af_majo + af_forf
        return period, af_base * 0

In [ ]:
from numpy import round

# Suppression du complément familial

class cf(ReformeRevenuDeBase.Variable):
    reference = Familles.column_by_name['cf']

    def function(self, simulation, period):
        '''
        L'allocation de base de la paje n'est pas cumulable avec le complément familial
        '''
        period = period.this_month
        paje_base = simulation.calculate('paje_base', period)
        apje_temp = simulation.calculate('apje_temp', period)
        ape_temp = simulation.calculate('ape_temp', period)
        cf_montant = simulation.calculate('cf_montant', period)
        residence_mayotte = simulation.calculate('residence_mayotte', period)

        cf_brut = not_(paje_base) * (apje_temp <= cf_montant) * (ape_temp <= cf_montant) * cf_montant
        # return period, not_(residence_mayotte) * round(cf_brut, 2)
        return period, not_(residence_mayotte) * round(cf_brut, 2) * 0

In [ ]:

# Suppression de l'allocation de rentrée scolaire

class ars(ReformeRevenuDeBase.Variable):
    reference = Familles.column_by_name['ars']

    def function(self, simulation, period):
        '''
        Allocation de rentrée scolaire brute de CRDS
        '''
        period_br = period.this_year
        return period_br, self.zeros()

In [ ]:
# Suppression du nombre d'enfants dans le calcul du RSA forfait logement

class rmi_nbp(ReformeRevenuDeBase.Variable):
    reference = Familles.column_by_name['rmi_nbp']

    def function(self, simulation, period):
        period = period.start.offset('first-of', 'month').period('month')
        age_holder = simulation.compute('age', period)
        smic55_holder = simulation.compute('smic55', period)
        nb_par = simulation.calculate('nb_par', period)
        P = simulation.legislation_at(period.start).minim.rmi

        age = self.split_by_roles(age_holder, roles = ENFS)
        smic55 = self.split_by_roles(smic55_holder, roles = ENFS)

        return period, nb_par # + nb_enf(age, smic55, 0, P.age_pac - 1)


In [ ]:
# Suppression de la cotisation patronale famille 

class famille(ReformeRevenuDeBase.Variable):
    reference = Individus.column_by_name['famille']

    def function(self, simulation, period):
        # cotisation = apply_bareme(
        #    simulation,
        #    period,
        #    cotisation_type = 'employeur',
        #    bareme_name = 'famille',
        #    variable_name = self.__class__.__name__,
        #    )
        #return period, cotisation        
        return period, self.zeros()

In [ ]:
# Baisse de l'éxonération Fillon

# /!\ CHANGER LES PARAMÈTRES DE L'ÉXONÉRATION FILLON (-5,25%)

#@ReformeRevenuDeBase.formula
#def taux_exo_fillon(ratio_smic_salaire, majoration, P):
#    '''
#    Exonération Fillon
#    http://www.securite-sociale.fr/comprendre/dossiers/exocotisations/exoenvigueur/fillon.htm
#    '''
#    # La divison par zéro engendre un warning
#    # Le montant maximum de l’allègement dépend de l’effectif de l’entreprise.
#    # Le montant est calculé chaque année civile, pour chaque salarié ;
#    # il est égal au produit de la totalité de la rémunération annuelle telle
#    # que visée à l’article L. 242-1 du code de la Sécurité sociale par un
#    # coefficient.
#    # Ce montant est majoré de 10 % pour les entreprises de travail temporaire
#    # au titre des salariés temporaires pour lesquels elle est tenue à
#    # l’obligation d’indemnisation compensatrice de congés payés.

#    Pf = P.exo_bas_sal.fillon
#    seuil = Pf.seuil
#    tx_max = (Pf.tx_max * not_(majoration) + Pf.tx_max2 * majoration) - 0.0525
#    if seuil <= 1:
#        return 0
#    # règle d'arrondi: 4 décimales au dix-millième le plus proche
#    taux_fillon = round_(tx_max * min_(1, max_(seuil * ratio_smic_salaire - 1, 0) / (seuil - 1)), 4)
#    return taux_fillon

In [ ]:
-

In [ ]:
# Création d'un revenu de base enfant - Version individus

class rdb_enf(ReformeRevenuDeBase.Variable):
    column = FloatCol
    entity_class = Individus
    label = u"Revenu de base enfant"

    def function(self, simulation, period):
        period = period.start.offset('first-of', 'month').period('month')
        age = simulation.calculate('age')
        P = simulation.legislation_at(period.start).fam.af        
        bmaf = P.bmaf
        
        return period, ((age < 14) * 0.41 + not_(age < 14) * 0.57) * bmaf * (age <= 18)

In [ ]:
class revdisp(ReformeRevenuDeBase.Variable):
    reference = Menages.column_by_name['revdisp']

    def function(self, simulation, period):
        '''
        Revenu disponible - ménage
        'men'
        '''
        period = period.start.period('year').offset('first-of')
        rev_trav_holder = simulation.compute('rev_trav', period)
        pen_holder = simulation.compute('pen', period)
        rev_cap_holder = simulation.compute('rev_cap', period)
        psoc_holder = simulation.compute('psoc', period)
        ppe_holder = simulation.compute('ppe', period)
        impo = simulation.calculate('impo', period)
        rdb_enfant_holder = simulation.calculate_add('rdb_enf', period)

        pen = self.sum_by_entity(pen_holder)
        ppe = self.cast_from_entity_to_role(ppe_holder, role = VOUS)
        ppe = self.sum_by_entity(ppe)
        psoc = self.cast_from_entity_to_role(psoc_holder, role = CHEF)
        psoc = self.sum_by_entity(psoc)
        rev_cap = self.sum_by_entity(rev_cap_holder)
        rev_trav = self.sum_by_entity(rev_trav_holder)
        rdb_enfant = self.sum_by_entity(rdb_enfant_holder)

        return period, rev_trav + pen + rev_cap + psoc + ppe + impo + rdb_enfant

# Tests

In [ ]:
reform = ReformeRevenuDeBase()

In [ ]:
scenario = tax_benefit_system.new_scenario().init_single_entity(
    period = 2014,
    parent1 = dict(
        birth = date(1980, 1, 1),
        salaire_imposable = 12000,
        statmarit = u'Marié',
        ),
    parent2 = dict(
        birth = date(1980, 1, 1),
        salaire_imposable = 46000,
        statmarit = u'Marié',
        ),
    enfants = [
        dict(
            birth = date(2010, 1, 1),
            ),
        dict(
            birth = date(2005, 1, 1),
            ),
        dict(
            birth = date(1999, 1, 1),
            ),
        ],
    )
simulation = scenario.new_simulation(debug = True)

In [ ]:
simulation_pauvre = tax_benefit_system.new_scenario().init_single_entity(
    period = 2014,
    parent1 = dict(
        birth = date(1980, 1, 1),
        salaire_imposable = 12000,
        statmarit = u'Marié',
        ),
    parent2 = dict(
        birth = date(1980, 1, 1),
        salaire_imposable = 6000,
        statmarit = u'Marié',
        ),
    enfants = [
        dict(
            birth = date(2010, 1, 1),
            ),
        dict(
            birth = date(2005, 1, 1),
            ),
        dict(
            birth = date(1999, 1, 1),
            ),
        ],
    ).new_simulation(debug = True)

In [ ]:
reform_simulation = reform.new_scenario().init_single_entity(
    period = 2014,
    parent1 = dict(
        birth = date(1980, 1, 1),
        salaire_imposable = 12000,
        statmarit = u'Marié',
        ),
    parent2 = dict(
        birth = date(1980, 1, 1),
        salaire_imposable = 46000,
        statmarit = u'Marié',
        ),
    enfants = [
        dict(
            birth = date(2010, 1, 1),
            ),
        dict(
            birth = date(2005, 1, 1),
            ),
        dict(
            birth = date(1999, 1, 1),
            ),
        ],
    ).new_simulation(debug = True)

In [ ]:
reform_simulation_pauvre = reform.new_scenario().init_single_entity(
    period = 2014,
    parent1 = dict(
        birth = date(1980, 1, 1),
        salaire_imposable = 12000,
        statmarit = u'Marié',
        ),
    parent2 = dict(
        birth = date(1980, 1, 1),
        salaire_imposable = 6000,
        statmarit = u'Marié',
        ),
    enfants = [
        dict(
            birth = date(2010, 1, 1),
            ),
        dict(
            birth = date(2005, 1, 1),
            ),
        dict(
            birth = date(1999, 1, 1),
            ),
        ],
    ).new_simulation(debug = True)

In [ ]:
simulation.calculate('nbptr')

In [ ]:
reform_simulation.calculate('nbptr')

In [ ]:
simulation.calculate_add('af')

In [ ]:
reform_simulation.calculate_add('af')

In [ ]:
simulation.calculate_add('cf')

In [ ]:
reform_simulation.calculate_add('cf')

In [ ]:
simulation.calculate('ars')

In [ ]:
reform_simulation.calculate_add('ars')

In [ ]:
simulation.calculate_add('rsa')

In [ ]:
from openfisca_core import web_tools

In [ ]:
print web_tools.get_trace_tool_link(scenario, ['rsa'])


In [ ]:
reform_simulation.calculate_add('rsa')

In [ ]:
simulation.calculate_add('rmi_nbp')

In [ ]:
reform_simulation.calculate_add('rmi_nbp')

In [ ]:
simulation.calculate('famille')

In [ ]:
reform_simulation.calculate('famille')

In [ ]:
reform_simulation.calculate_add('rdb_enfant_famille')

In [ ]:
reform_simulation.calculate_add('rdb_enf')

In [ ]:
reform_simulation.calculate('csgenf')

In [ ]:
simulation.calculate('csg')

In [ ]:
reform_simulation.calculate('csg')

In [ ]:
simulation.calculate('revdisp')

In [ ]:
reform_simulation.calculate('revdisp')

In [ ]:
simulation_pauvre.calculate('revdisp')

In [ ]:
reform_simulation_pauvre.calculate('revdisp')

In [ ]:
simulation.calculate('salaire_imposable')

In [ ]:
reform_simulation.calculate('salaire_imposable')

In [ ]:
simulation.calculate('salaire_super_brut')

In [ ]:
reform_simulation.calculate('salaire_super_brut')

In [ ]:
simulation.calculate('salaire_net')

In [ ]:
reform_simulation.calculate('salaire_net')

In [ ]:
simulation.calculate('pfam')

In [ ]:
reform_simulation.calculate('pfam')

In [ ]:
simulation_pauvre.calculate('psoc')

In [ ]:
simulation.calculate('cotisations_salariales')

In [ ]:
reform_simulation_pauvre.calculate('pfam')

In [ ]:
simulation.calculate('irpp')

In [ ]:
reform_simulation.calculate('irpp')

In [ ]:
simulation.calculate('allegement_fillon')